#### Fake News Recognition  

In [9]:
import pandas as pd
import numpy as np
import re
import string

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier


In [10]:
df_fake = pd.read_csv(r"D:\Open Classroom\Datasets\Fake_news\Fake.csv", error_bad_lines=False)
df_true = pd.read_csv(r"D:\Open Classroom\Datasets\Fake_news\True.csv")

In [11]:
df_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [12]:
df_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [13]:
df_fake.shape, df_true.shape

((23481, 4), (21417, 4))

In [14]:
df_fake.loc[4, "text"]

'Pope Francis used his annual Christmas Day message to rebuke Donald Trump without even mentioning his name. The Pope delivered his message just days after members of the United Nations condemned Trump s move to recognize Jerusalem as the capital of Israel. The Pontiff prayed on Monday for the  peaceful coexistence of two states within mutually agreed and internationally recognized borders. We see Jesus in the children of the Middle East who continue to suffer because of growing tensions between Israelis and Palestinians,  Francis said.  On this festive day, let us ask the Lord for peace for Jerusalem and for all the Holy Land. Let us pray that the will to resume dialogue may prevail between the parties and that a negotiated solution can finally be reached. The Pope went on to plead for acceptance of refugees who have been forced from their homes, and that is an issue Trump continues to fight against. Francis used Jesus for which there was  no place in the inn  as an analogy. Today, as

In [15]:
df_fake["class"] = 0
df_true["class"] = 1

In [17]:
df_merge = pd.concat([df_true, df_fake])
df_merge.to_csv("D:\Open Classroom\Datasets\Fake_news\Fake_news_combined_dataset.csv")

In [19]:
df = pd.read_csv("D:\Open Classroom\Datasets\Fake_news\Fake_news_combined_dataset.csv")
df.drop(columns = ["Unnamed: 0"], inplace = True)
df.head()

,title,text,subject,date,class
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [20]:
df = df.drop(["title", "subject", "date"], axis = 1)
df.sample(frac = 2, replace = True).head()

,text,class
29082,He s not wrong.Demonstrating that he is the on...,0
7597,WASHINGTON (Reuters) - Twelve U.S. senators ur...,1
20271,WASHINGTON (Reuters) - U.S. President Donald T...,1
18747,TOKYO (Reuters) - Even as a new party under a ...,1
17875,NAIROBI (Reuters) - Kenyan President Uhuru Ken...,1


In [21]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [24]:
#Drop all special characters and unused words

def word_drop(text):
    text = text.lower()
    text = re.sub("\[.*?\]", " ", text)
    text = re.sub("\\W", " ", text)
    text = re.sub("http?://\S+|www.\S+", " ", text)
    text = re.sub("<.*?>+", " ", text)
    text = re.sub("[%s]" % re.escape(string.punctuation), " ", text)
    text = re.sub("\n", " ", text)
    text = re.sub("\w*\d\w*", " ", text)
    return text


In [25]:
df["text"] = df["text"].apply(word_drop)
df.sample(frac = 1).head()

,text,class
2189,beijing reuters china said on tuesday that...,1
10875,washington reuters the head of the u s se...,1
41969,watch juan williams make several attempts to s...,0
23066,a student at transylvania university was injur...,0
6217,washington reuters u s president elect do...,1


In [26]:
df.sample(frac = 1).head()

,text,class
34408,hillary clinton ally david brock is offering t...,0
3940,washington reuters an expected proposal fr...,1
1777,washington reuters an ideologically divide...,1
2189,beijing reuters china said on tuesday that...,1
37691,mccain our intelligence agencies concluded un...,0


In [27]:
X = df["text"]
y = df["class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

vectorization = TfidfVectorizer()

Xvect_train = vectorization.fit_transform(X_train)
Xvect_test = vectorization.transform(X_test)


#### Logistic Regresssion

In [28]:
LR = LogisticRegression()

LR.fit(Xvect_train, y_train)

LogisticRegression()

In [29]:
model_accuracy = LR.score(Xvect_test, y_test)
print(f"Accuracy Score : {round(model_accuracy*100, 2)} %")

Accuracy Score : 98.58 %


In [30]:
model_pred = LR.predict(Xvect_test)
print(f" Test Prediction {model_pred}")

 Test Prediction [1 1 1 ... 0 0 1]


In [31]:
report = classification_report(y_test, model_pred)
print(report)

              precision    recall  f1-score   support

           0       0.99      0.98      0.99      5925
           1       0.98      0.99      0.99      5300

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



#### Gradient Boosting Classifier

In [32]:
gbc = GradientBoostingClassifier(random_state = 0)
gbc.fit(Xvect_train, y_train)

GradientBoostingClassifier(random_state=0)

In [33]:
gbc.score(Xvect_test, y_test)

0.9951002227171493

In [34]:
gbc_pred = gbc.predict(Xvect_test)

In [35]:
print(classification_report(y_test, gbc_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5925
           1       0.99      1.00      0.99      5300

    accuracy                           1.00     11225
   macro avg       0.99      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



#### Random Forest Classifier

In [36]:
rfc = RandomForestClassifier(random_state = 0)
rfc.fit(Xvect_train, y_train)

RandomForestClassifier(random_state=0)

In [37]:
rfc.score(Xvect_test, y_test)

0.9893095768374165

In [38]:
rfc_pred = rfc.predict(Xvect_test)

In [39]:
print(classification_report(rfc_pred, y_test))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5943
           1       0.99      0.99      0.99      5282

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



#### Manual Testing

In [40]:
def output_label(news):
    if news == 0:
        return "Fake / Forgery News"
    elif news == 1:
        return "Genuine / Original News"

def manual_testing(bulletin):
    testing_news = {"Headlines": [bulletin]}
    new_df_test = pd.DataFrame(testing_news)
    new_df_test["Headlines"] = new_df_test["Headlines"].apply(word_drop)
    new_x_test = new_df_test["Headlines"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_GBC = gbc.predict(new_xv_test)
    pred_RFC = rfc.predict(new_xv_test)
    
    #print("\n\n\n", new_df_test.loc[0, "Headlines"])
    return (print(f'\n\n\nFake News Detection Application \nPredictive Model Outcomes\n\nLogistic Regression: {output_label(pred_LR)} \nGradient Boosting Classifier: {output_label(pred_GBC)} \nRandom Forest Classifier: {output_label(pred_RFC)}'))
        

In [41]:
headline = str(input()).title()
manual_testing(headline)

 chandigarh  farmers on sunday blocked roads in haryana and burnt effigies of prime minister narendra modi in neighbouring punjab  protesting against the farm bills passed in parliament two mlas from the jannayak janta party   jjp    which is part of the bjp led ruling coalition in haryana  also participated in the protests  reflecting a rift on the issue among their ranks farmers occupied portions of roads and highways at several places in the state between   and   pm  following a call given by the haryana unit of the bhartiya kisan union against the agri marketing bills  two of which were passed by rajya sabha on sunday after   pm  reports said the road blockades were lifted  ending the day s protests in which bku was also supported by several other farmer organisations in ambala   haryana police used a water cannon at the state s border to stop punjab youth congress workers from entering the district and travelling ahead to delhi as part of their  tractor rally  the punjab protester




Fake News Detection Application 
Predictive Model Outcomes

Logistic Regression: Genuine / Original News 
Gradient Boosting Classifier: Fake / Forgery News 
Random Forest Classifier: Genuine / Original News
